In [1]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot
from pandas import Series, datetime
from pandas.tools.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
import seaborn as sns

/Users/mhafizhussin/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [7]:
# Loading the dataset to a dataframe
price_df = pd.read_csv('cimb_clean.csv', parse_dates=[0])
news_df = pd.read_csv('cimb_news.csv', parse_dates=[0])

In [8]:
print(price_df.shape)
print(price_df.dtypes)
print(news_df.shape)
print(news_df.dtypes)

(239, 11)
Date          datetime64[ns]
High                 float64
Low                  float64
Open                 float64
Close                float64
Volume               float64
yhat                 float64
us_china             float64
trump                float64
malaysia             float64
target_cls           float64
dtype: object
(74, 3)
Date     datetime64[ns]
Title            object
Score             int64
dtype: object


In [18]:
merged_dataframe = news_df[['Date', 'Score']].merge(price_df, how='inner', on='Date', left_index=True)
print(merged_dataframe.shape)
merged_dataframe.head()

(56, 12)


,Date,Score,High,Low,Open,Close,Volume,yhat,us_china,trump,malaysia,target_cls
232,2017-12-07,1,6.04,5.97,6.00,5.97,23724000.0,6.059920,43.571429,13.714286,47.428571,0.0
231,2017-12-06,0,6.02,5.95,5.95,5.97,21382900.0,6.038819,41.428571,14.285714,47.571429,0.0
227,2017-11-30,0,6.09,5.94,5.94,6.05,47423000.0,5.970498,40.571429,16.857143,44.571429,0.0
225,2017-11-28,1,5.97,5.93,5.95,5.93,11081800.0,5.948344,44.285714,17.428571,42.285714,1.0
222,2017-11-23,0,6.06,5.95,5.97,5.98,6929200.0,5.952610,53.571429,17.142857,39.571429,0.0


In [21]:
merged_dataframe = price_df.merge(news_df[['Date','Score']], how='left', on='Date',left_index=True)
print(merged_dataframe.shape)
merged_dataframe.head()

(244, 12)


,Date,High,Low,Open,Close,Volume,yhat,us_china,trump,malaysia,target_cls,Score
73,2017-01-02,4.51,4.51,4.51,4.51,0.0,4.422928,62.0,58.428571,39.428571,1.0,NaN
73,2017-01-03,4.57,4.51,4.53,4.54,12056200.0,4.438930,64.0,61.857143,39.857143,1.0,NaN
73,2017-01-04,4.62,4.54,4.54,4.60,19551700.0,4.461632,66.0,65.285714,40.285714,1.0,NaN
73,2017-01-05,4.67,4.60,4.62,4.66,20520700.0,4.490498,68.0,68.714286,40.714286,1.0,NaN
73,2017-01-06,4.74,4.66,4.66,4.74,18234800.0,4.515630,70.0,72.142857,41.142857,1.0,NaN


In [26]:
merged_dataframe['Score'].fillna(0, inplace=True)
print(merged_dataframe.shape)

(244, 12)


,Date,High,Low,Open,Close,Volume,yhat,us_china,trump,malaysia,target_cls,Score
73,2017-01-02,4.51,4.51,4.51,4.51,0.0,4.422928,62.000000,58.428571,39.428571,1.0,0.0
73,2017-01-03,4.57,4.51,4.53,4.54,12056200.0,4.438930,64.000000,61.857143,39.857143,1.0,0.0
73,2017-01-04,4.62,4.54,4.54,4.60,19551700.0,4.461632,66.000000,65.285714,40.285714,1.0,0.0
73,2017-01-05,4.67,4.60,4.62,4.66,20520700.0,4.490498,68.000000,68.714286,40.714286,1.0,0.0
73,2017-01-06,4.74,4.66,4.66,4.74,18234800.0,4.515630,70.000000,72.142857,41.142857,1.0,0.0
73,2017-01-09,4.79,4.74,4.75,4.76,13204800.0,4.534182,72.428571,81.714286,41.142857,1.0,0.0
73,2017-01-10,4.80,4.75,4.76,4.80,15809700.0,4.546839,70.857143,84.428571,40.285714,1.0,0.0
73,2017-01-11,4.83,4.80,4.81,4.82,19984100.0,4.567585,69.285714,87.142857,39.428571,1.0,0.0
73,2017-01-12,4.87,4.82,4.82,4.86,23162100.0,4.585673,67.714286,89.857143,38.571429,0.0,0.0
73,2017-01-13,4.88,4.81,4.85,4.82,11672300.0,4.591469,66.142857,92.571429,37.714286,0.0,0.0


In [74]:
merged_dataframe.dropna().head()

,Date,High,Low,Open,Close,Volume,yhat,us_china,trump,malaysia,Score,target_cls
0,2017-01-02,4.51,4.51,4.51,4.51,0.0,4.422928,62.0,58.428571,39.428571,0.0,1.0
1,2017-01-03,4.57,4.51,4.53,4.54,12056200.0,4.438930,64.0,61.857143,39.857143,0.0,1.0
2,2017-01-04,4.62,4.54,4.54,4.60,19551700.0,4.461632,66.0,65.285714,40.285714,0.0,1.0
3,2017-01-05,4.67,4.60,4.62,4.66,20520700.0,4.490498,68.0,68.714286,40.714286,0.0,1.0
4,2017-01-06,4.74,4.66,4.66,4.74,18234800.0,4.515630,70.0,72.142857,41.142857,0.0,1.0


In [61]:
# Push the Label column to the end of the dataframe
cols = list(merged_dataframe)
print(cols)
cols.append(cols.pop(cols.index('target_cls')))
merged_dataframe = merged_dataframe.loc[:, cols]
merged_dataframe.head()

['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'yhat', 'us_china', 'trump', 'malaysia', 'Score', 'target_cls']


,Date,High,Low,Open,Close,Volume,yhat,us_china,trump,malaysia,Score,target_cls
0,2017-01-02,4.51,4.51,4.51,4.51,0.0,4.422928,62.0,58.428571,39.428571,0.0,1.0
1,2017-01-03,4.57,4.51,4.53,4.54,12056200.0,4.438930,64.0,61.857143,39.857143,0.0,1.0
2,2017-01-04,4.62,4.54,4.54,4.60,19551700.0,4.461632,66.0,65.285714,40.285714,0.0,1.0
3,2017-01-05,4.67,4.60,4.62,4.66,20520700.0,4.490498,68.0,68.714286,40.714286,0.0,1.0
4,2017-01-06,4.74,4.66,4.66,4.74,18234800.0,4.515630,70.0,72.142857,41.142857,0.0,1.0


In [68]:
# Separate the dataframe for input(X) and output variables(y)
# merged_dataframe = merged_dataframe.dropna()
X = merged_dataframe.loc[:,'High':'Score']
y = merged_dataframe.loc[:,'target_cls']
# Set the validation size, i.e the test set to 20%
validation_size = 0.20
# Split the dataset to test and train sets
# Split the initial 70% of the data as training set and the remaining 30% data as the testing set
train_size = int(len(X.index) * 0.7)
print(len(y))
print(train_size)


243
170


In [69]:
X_train, X_test = X.loc[0:train_size, :], X.loc[train_size: len(X.index), :]
y_train, y_test = y[0:train_size+1], y.loc[train_size: len(X.index)]
print('Observations: %d' % (len(X.index)))
print('X Training Observations: %d' % (len(X_train.index)))
print('X Testing Observations: %d' % (len(X_test.index)))
print('y Training Observations: %d' % (len(y_train)))
print('y Testing Observations: %d' % (len(y_test)))

Observations: 243
X Training Observations: 171
X Testing Observations: 73
y Training Observations: 171
y Testing Observations: 73


In [83]:
merged_dataframe.to_csv('merged_df.csv', index=False)

In [70]:
# MOdels
num_folds = 10
scoring = 'accuracy'
# Append the models to the models list
models = []
models.append(('LR' , LogisticRegression()))
models.append(('LDA' , LinearDiscriminantAnalysis()))
models.append(('KNN' , KNeighborsClassifier()))
models.append(('CART' , DecisionTreeClassifier()))
models.append(('NB' , GaussianNB()))
models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
models.append(('XGBoost', XGBClassifier()))

In [72]:
# Evaluate each algorithm for accuracy
results = []
names = []
'''
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) '''

for name, model in models:
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accu_score = accuracy_score(y_test, y_pred)
    print(name + ": " + str(accu_score))

LR: 0.5753424657534246
LDA: 0.3698630136986301
KNN: 0.4794520547945205
CART: 0.5342465753424658
NB: 0.4520547945205479
SVM: 0.6027397260273972


/Users/mhafizhussin/anaconda2/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


RF: 0.4520547945205479
XGBoost: 0.4383561643835616


In [77]:
# The best model = SVM, 
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model_svm = SVC()
model_svm.fit(rescaledX, y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_test)
predictions = model_svm.predict(rescaledValidationX)
print("accuracy score:")
print(accuracy_score(y_test, predictions))
print("confusion matrix: ")
print(confusion_matrix(y_test, predictions))
print("classification report: ")
print(classification_report(y_test, predictions))

model_xgb = XGBClassifier()
model_xgb.fit(rescaledX, y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_test)
predictions = model_xgb.predict(rescaledValidationX)
print("accuracy score:")
print(accuracy_score(y_test, predictions))
print("confusion matrix: ")
print(confusion_matrix(y_test, predictions))
print("classification report: ")
print(classification_report(y_test, predictions))

accuracy score:
0.4657534246575342
confusion matrix: 
[[16 27]
 [12 18]]
classification report: 
              precision    recall  f1-score   support

         0.0       0.57      0.37      0.45        43
         1.0       0.40      0.60      0.48        30

   micro avg       0.47      0.47      0.47        73
   macro avg       0.49      0.49      0.47        73
weighted avg       0.50      0.47      0.46        73

accuracy score:
0.4383561643835616
confusion matrix: 
[[15 28]
 [13 17]]
classification report: 
              precision    recall  f1-score   support

         0.0       0.54      0.35      0.42        43
         1.0       0.38      0.57      0.45        30

   micro avg       0.44      0.44      0.44        73
   macro avg       0.46      0.46      0.44        73
weighted avg       0.47      0.44      0.44        73



In [78]:
# Scaling Random Forests
model_rf = RandomForestClassifier(n_estimators=1000)
model_rf.fit(rescaledX, y_train)
# estimate accuracy on validation dataset
rescaledValidationX = scaler.transform(X_test)
predictions = model_rf.predict(rescaledValidationX)
print("accuracy score:")
print(accuracy_score(y_test, predictions))
print("confusion matrix: ")
print(confusion_matrix(y_test, predictions))
print("classification report: ")
print(classification_report(y_test, predictions))

accuracy score:
0.4520547945205479
confusion matrix: 
[[ 9 34]
 [ 6 24]]
classification report: 
              precision    recall  f1-score   support

         0.0       0.60      0.21      0.31        43
         1.0       0.41      0.80      0.55        30

   micro avg       0.45      0.45      0.45        73
   macro avg       0.51      0.50      0.43        73
weighted avg       0.52      0.45      0.41        73



In [79]:
# XGBoost on Stock Price dataset, Tune n_estimators and max_depth
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib

matplotlib.use('Agg')
model = XGBClassifier()
n_estimators = [150, 200, 250, 450, 500, 550, 1000]
max_depth = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
print(max_depth)
best_depth = 0
best_estimator = 0
max_score = 0
for n in n_estimators:
    for md in max_depth:
        model = XGBClassifier(n_estimators=n, max_depth=md)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        if score > max_score:
            max_score = score
            best_depth = md
            best_estimator = n
        print("Score is " + str(score) + " at depth of " + str(md) + " and estimator " + str(n))
print("Best score is " + str(max_score) + " at depth of " + str(best_depth) + " and estimator of " + str(best_estimator))

/Users/mhafizhussin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/mhafizhussin/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/mhafizhussin/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/mhafizhussin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mhafizhussin/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/mhafizhussin/anaconda2/lib/python2.7/s

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Score is 0.4794520547945205 at depth of 1 and estimator 150
Score is 0.4520547945205479 at depth of 2 and estimator 150
Score is 0.4520547945205479 at depth of 3 and estimator 150
Score is 0.4520547945205479 at depth of 4 and estimator 150
Score is 0.4657534246575342 at depth of 5 and estimator 150
Score is 0.4520547945205479 at depth of 6 and estimator 150
Score is 0.4246575342465753 at depth of 7 and estimator 150
Score is 0.4794520547945205 at depth of 8 and estimator 150
Score is 0.4657534246575342 at depth of 9 and estimator 150
Score is 0.4657534246575342 at depth of 10 and estimator 150
Score is 0.4520547945205479 at depth of 11 and estimator 150
Score is 0.4520547945205479 at depth of 12 and estimator 150
Score is 0.4794520547945205 at depth of 1 and estimator 200
Score is 0.4246575342465753 at depth of 2 and estimator 200
Score is 0.4657534246575342 at depth of 3 and estimator 200
Score is 0.4931506849315068 at depth of 4 and estimator 2

In [80]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X)
transformed = pca.transform(X)

transformed.shape
print(type(transformed))

<type 'numpy.ndarray'>


In [81]:
pca_df = pd.DataFrame(transformed)

X_train_pca, X_test_pca = pca_df.loc[0:train_size, :], pca_df.loc[train_size: len(X.index), :]

clf = XGBClassifier(n_estimators=500, max_depth=3)
clf.fit(X_train_pca, y_train)
y_pred_pca = clf.predict(X_test_pca)
score = accuracy_score(y_test, y_pred_pca)
print("Score is "+ str(score))

Score is 0.4383561643835616
